In [34]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import csv
from datetime import datetime as dt
from typing import List, Tuple, Dict, Any
from datetime import datetime as dt
from typing import Callable

DEBUG = True 

def debugger_factory(show_args=True) -> Callable:
    def debugger(func: Callable) -> Callable:
        if DEBUG:   
            def wrapper(*args, **kwargs):
                if show_args:
                    print(f'{func.__name__} was called with args={args}, kwargs={kwargs}')  
                t0 = dt.now()                              
                results = func(*args, **kwargs)            
                print(f'{func.__name__} ran for {dt.now()-t0}')  
                return results                             
            return wrapper
        return func                                    
    return debugger
HALF_DECK_SIZE = 26

def get_init_deck(half_deck_size: int) -> np.ndarray:
    """
    This function creates an initial decks of 0s (Blacks) and 1s (Reds)
    
    """
    return np.array([0] * half_deck_size + [1] * half_deck_size)

def shuffle_deck(seed: int, deck: np.ndarray) -> np.ndarray:
    """
    This function shuffles a deck using a specific random seed. Shuffles a given deck using a specified seed.
    
    """
    np.random.seed(seed)
    shuffled_deck = deck.copy()
    np.random.shuffle(shuffled_deck)
    return shuffled_deck

@debugger_factory()    
def get_n_decks(num_decks: int, num_cards: int = HALF_DECK_SIZE) -> List[Tuple[int, np.ndarray]]:
    """
    This function creates a list of shuffled decks each with unique random seeds. Also, each deck has a tuple containing (seed, deck).
    """
    init_deck = get_init_deck(num_cards)
    decks = []
    for _ in range(num_decks):
        seed = np.random.randint(0, 2**32-1)
        shuffled_deck = shuffle_deck(seed, init_deck)
        decks.append((seed, shuffled_deck))
    return decks
    

def generate_sequences() -> List[Tuple[str, str, str]]:
     """This function creates all possible combinations of 0s and 1s Generates all possible three-card sequences of Bs and Rs."""
     return list(itertools.product('BR', repeat=3))


def load_results_from_csv(filepath: str) -> Dict[Tuple[str, str], Dict[str, float]]:
    """
    This function allows us to load the results_results.csv file containing all the results from multiple games of Penney.
    This function is only used if we want augment existing data.
    """
    results = {}
    with open(filepath, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            p1 = tuple(row["Player 1"])  
            p2 = tuple(row["Player 2"])  
            if len(p1) != 3 or len(p2) != 3:
                print(f"Skipping invalid row: {row}")
                continue
            results[(p1, p2)] = {
                "Player 2 Win % (Trick)": float(row["Player 2 Win % (Trick)"]),
                "Player 2 Win % (Total)": float(row["Player 2 Win % (Total)"]),
                "Draw % (Trick)": float(row["Draw % (Trick)"]),
                "Draw % (Total)": float(row["Draw % (Total)"])
            }
    return results
def load_seeds_from_csv(filepath: str) -> List[int]:
    """
    This function allows us to load the results_seeds.csv file containg all the seeds used for each deck.
    This function is only used if we want to augment existing data.
    """
    seeds = []
    with open(filepath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  
        for row in reader:
            seeds.append(int(row[0]))
    return seeds


In [35]:


def generate_1_game(deck: np.ndarray, player1_seq: Tuple[str, str, str], player2_seq: Tuple[str, str, str]) -> Tuple[int, int, int, int]:
    """
    Simulates one game of Penney.
    - Players choose their sequences.
    - Flip cards one at a time until a player's sequence occurs.
    - The player who gets a match is awarded one trick and all previous cards plus the 3 matching cards.
    """

    #This line of code converts each player's sequences into strings.
    p1_str, p2_str = ''.join(player1_seq), ''.join(player2_seq)
    #These two lines of code help us keep track of how many tricks and cards each player won.
    p1_tricks, p2_tricks = 0, 0
    p1_total_cards, p2_total_cards = 0, 0
    #This will allow us to keep track of the number of cards flipped before a sequence match.
    collected_cards = 0
    #This will us to keep track of the running sequence of the flipped cards.
    current_sequence = []
    
    #This loop allows us to play the game of Penney by flipping one card at a time. Also, keeps track of number of tricks and total cards won for each player.
    for card in deck:
        current_sequence.append('B' if card == 0 else 'R')
        collected_cards += 1

        if len(current_sequence) >= 3:
            trick_str = ''.join(current_sequence[-3:])

            if trick_str == p1_str:
                p1_tricks += 1
                p1_total_cards += collected_cards
                collected_cards = 0
                current_sequence = []
                continue

            if trick_str == p2_str:
                p2_tricks += 1
                p2_total_cards += collected_cards
                collected_cards = 0
                current_sequence = []
                continue

    return p1_tricks, p2_tricks, p1_total_cards, p2_total_cards

@debugger_factory()

def simulate_games(
    num_decks: int,
    existing_results: Dict[Tuple, Dict[str, float]] = None,
    existing_seeds: List[int] = None,
    save_prefix: str = "results",
    save_to_file: bool = True
) -> Tuple[Dict[Tuple, Dict[str, float]], List[int]]:
    """
    Simulates multiple games of Penney. This function can also augmenting existing results if you so choose. 
    Outputs the results and seeds used.
    """
    #This will generate all possible 3-letter red/black sequences.
    sequences = generate_sequences()

    # If there is no existing data, then it will initialize new dictionaries. Otherwise, it will use the existing results and seeds.
    if existing_results is None:
        results = {(p1, p2): {"Player 2 Win % (Trick)": 0.0,
                              "Player 2 Win % (Total)": 0.0,
                              "Draw % (Trick)": 0.0,
                              "Draw % (Total)": 0.0}
                   for p1 in sequences for p2 in sequences}
        used_seeds = []
    else:
        results = existing_results
        used_seeds = existing_seeds.copy() if existing_seeds else []
        existing_total = len(used_seeds)
        for key in results:
            for metric in results[key]:
                results[key][metric] *= existing_total
        

    #This will allow us to generate new decks and seeds.
    decks = get_n_decks(num_decks)
    
    #These loops will allow us to loop over each deck. These loops will also allow us to play multiple games of Penney for each deck.
    #These loops will also allow us to determine the winner based on the number tricks or the number of total cards for each deck.
    for seed, deck in decks:
        used_seeds.append(seed)
        for p1 in sequences:
            for p2 in sequences:
                p1_tricks, p2_tricks, p1_cards, p2_cards = generate_1_game(deck, p1, p2)

                if (p1, p2) not in results:
                    results[(p1, p2)] = {
                        "Player 2 Win % (Trick)": 0.0,
                        "Player 2 Win % (Total)": 0.0,
                        "Draw % (Trick)": 0.0,
                        "Draw % (Total)": 0.0
                    }
                
                if p1_tricks > p2_tricks:
                    pass
                elif p1_tricks < p2_tricks:
                    results[(p1, p2)]["Player 2 Win % (Trick)"] += 1
                else:
                    results[(p1, p2)]["Draw % (Trick)"] += 1
                
                if p1_cards > p2_cards:
                    pass
                elif p1_cards < p2_cards:
                    results[(p1, p2)]["Player 2 Win % (Total)"] += 1
                else:
                    results[(p1, p2)]["Draw % (Total)"] += 1


    #These 4 lines of code will allow us to determine the probabilities.
    total_decks = len(used_seeds)
    for key in results:
        for metric in results[key]:
            results[key][metric] /= total_decks
    

    #These blocks of code will save our probabilities/results as a 'results_results.csv' file. 
    #These blocks of code will also save our used seeds as a 'results_seeds.csv' file
    if save_to_file:
        csv_filename = f"{save_prefix}_results.csv"
        with open(csv_filename, mode='w', newline='') as csvfile:
            fieldnames = ["Player 1", "Player 2", "Player 2 Win % (Trick)", "Player 2 Win % (Total)", "Draw % (Trick)", "Draw % (Total)"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for (p1, p2), stats in results.items():
                row = {
                    "Player 1": ''.join(p1),
                    "Player 2": ''.join(p2),
                    **stats
                }
                writer.writerow(row)

        seeds_filename = f"{save_prefix}_seeds.csv"
        with open(seeds_filename, mode='w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Seed"])
            for seed in used_seeds:
                writer.writerow([seed])

    return results, used_seeds



In [36]:

def visualize_heatmap(results: Dict[Tuple, Dict[str, float]], metrics: List[str], save_path: str = "heatmap", num_decks: int=1000000):
    """
    Visualizes win and draw percentages. Outputs two different heatmaps (one scoring by tricks and the other scoring by total cards).
    """
    #This will generate all possible 3-letter red/black sequences.
    sequences = generate_sequences()
    #This will allow us to convert each sequence from a tuple to a string when labeling the axes for our heatmaps.
    formatted_labels = [''.join(seq) for seq in sequences]

    #These three lines of code below allow us to title each of our heatmaps with the appropriate title.
    custom_titles = {
        "Player 2 Win % (Trick)": f"My Chance of Win(Draw)\nby Tricks\nN = {num_decks:,}",
        "Player 2 Win % (Total)": f"My Chance of Win(Draw)\nby Cards\nN = {num_decks:,}"
    }
    
    for idx, metric in enumerate(metrics):
        
        matrix = np.zeros((len(sequences), len(sequences)))
        annotations = np.empty((len(sequences), len(sequences)), dtype=object)
        mask = np.zeros_like(matrix, dtype=bool)

        
        for (p1_seq, p2_seq), stats in results.items():
            i, j = sequences.index(p1_seq), sequences.index(p2_seq)

            if i == j:  
                matrix[i, j] = np.nan  
                annotations[i, j] = ""  
                mask[i, j] = True  
            else:
                win_pct = round(stats.get(metric, 0) * 100)
                draw_metric = "Draw % (Trick)" if "Trick" in metric else "Draw % (Total)"
                draw_pct = int(stats.get(draw_metric, 0) * 100)
                annotations[i, j] = f"{win_pct} ({draw_pct})"
                matrix[i, j] = win_pct

        
        cmap = sns.color_palette("Blues", as_cmap=True) 
        cmap.set_bad(color='lightgrey')  

        
        fig, ax = plt.subplots(figsize=(10, 8))

        sns.heatmap(
            matrix,
            annot=annotations,
            fmt="",
            xticklabels=formatted_labels,
            yticklabels=formatted_labels,
            cmap=cmap,
            linewidths=0.5,
            annot_kws={"size": 9},
            mask=mask,  
            cbar=False,  
            ax=ax
        )

        #These two lines of code will label the axes.
        ax.set_xlabel("Player 1 Sequence", fontsize=14, labelpad=15)
        ax.set_ylabel("Player 2 Sequence", fontsize=14, labelpad=15)

        #These two lines of code will allow us to put ticks on our heatmaps.
        ax.set_xticklabels(formatted_labels, rotation=0, ha="center")
        ax.set_yticklabels(formatted_labels, rotation=0)

        #This line of code will allow us to use our custom titles when labeling our heatmaps.
        ax.set_title(custom_titles.get(metric, f"Win Probability by {metric}\nN = 1,000,000"), fontsize=16)

        #These lines of save each plot as a SVG file. One heatmap is based on the Tricks scoring method and the other heatmap is based
        #on the Total Cards scoring method. There should be a total of two SVG files.
        filename = f"{save_path}_{metric.replace(' ', '_')}.svg"
        plt.tight_layout()
        plt.savefig(filename, format="svg")
        print(f"Saved: {filename}")
        plt.close(fig)





In [37]:
#The two lines of commented code below is for when you want to create new decks from scratch without augmenting the existing data.
#Uncomment these two lines of code if you want to do so. Also, comment out the rest of the code below it.
#results1, seeds1 = simulate_games(num_decks=1000000)
#visualize_heatmap(results1, ["Player 2 Win % (Trick)", "Player 2 Win % (Total)"], num_decks=1000000)

#These two lines of code allows us to load our existing data for augmentation.
results_old = load_results_from_csv("results_results.csv")
seeds_old = load_seeds_from_csv("results_seeds.csv")

#These 4 lines of code below allows us to augment our existing data with any number of new decks. Simply change num_decks to whichever number you desire.
results_new, seeds_new = simulate_games(
    num_decks=100,
    existing_results=results_old,
    existing_seeds=seeds_old
)
#These two lines of code will allows us to update our heatmaps with the new decks.
total_decks = len(seeds_new)
visualize_heatmap(results_new, ["Player 2 Win % (Trick)", "Player 2 Win % (Total)"], num_decks=total_decks)

simulate_games was called with args=(), kwargs={'num_decks': 1000000}
get_n_decks was called with args=(1000000,), kwargs={}
get_n_decks ran for 0:00:14.660390
simulate_games ran for 0:25:48.960242
Saved: heatmap_Player_2_Win_%_(Trick).svg
Saved: heatmap_Player_2_Win_%_(Total).svg
